In [140]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import astropy.wcs
from astropy.io import fits
import astropy.io
from IPython.display import clear_output
from astropy.stats import sigma_clipped_stats
from photutils import datasets
from photutils import DAOStarFinder
from photutils import find_peaks
from astropy.visualization import SqrtStretch
from astropy.visualization.mpl_normalize import ImageNormalize
from photutils import CircularAperture
from photutils import RectangularAperture
from matplotlib.backends.backend_pdf import PdfPages

#df = pd.read_csv("simultaneous_event_candidates_table_MU+Tomo-e8.csv")
df = pd.read_csv("test8.csv")
angle = df["Angle"].values
x1 = df["x_1"].values
x2 = df["x_2"].values
X = df["X"].values
Y = df["Y"].values
rectangle_wide = (x2 - x1)/2
filename_xml = "d_xml_gaia/sub" + df["ID"] + "_gaia.xml"
filename_fits = "d_fits/sub" + df["ID"] + ".fits"
output_filename = "sub" + df["ID"] + "_star.csv"
with open("d_xml_gaia/sub571226_095_gaia.xml") as f:
    a = f.readlines()
    b = a[116]#<TABLEDATA>
    c = a[-5] #</TABLEDATA>

############################################################################
#xml_all = np.arange(6).reshape(6)
#xml_all = np.arange(5).reshape(5)
for i in range(len(filename_xml)):
#for i in range(1):
    print filename_xml[i],i
    xml_all = np.arange(9).reshape(9)
    xml_list = np.arange(5).reshape(5)
    pixel_sextractor = np.arange(2).reshape(2)
    
###########################################  Peak Detection #######
    fits = astropy.io.fits.open(filename_fits[i])
    hdul = fits
    data = hdul[0].data
    data = data[1,:,:]#差分前のフレームだけにする
    data = np.reshape(data, (282, 500))#2次元化
    wcs = astropy.wcs.WCS(fits[0].header, naxis = 2)
    """
    x = np.arange(x1[i], x2[i], 1)
    y = np.tan(np.deg2rad(90 - angle[i]))*x + Y[i] + np.tan(np.deg2rad(90 - angle[i]))*X[i]*(-1)
    y = y.astype(np.int64)
    y = y.reshape(1,len(y))
    y2 = y 
    for i in range(-10,11):
        b = y + i
        y2 = np.vstack((y2,b))
    y2 = np.delete(y2,0,0)
    #rotation_angle = np.tan(np.deg2rad(90 - angle[i])) #流星周りのマスク
    #rectangular = RectangularAperture((X[i],Y[i]), w=500, h=30., theta=rotation_angle)
    #masks = rectangular.to_mask(method='exact')
    #print data
    #mask = masks[0]
    #data_cutout_aper = mask.multiply(data)
    mask = np.zeros_like(data, dtype=bool)
    mask[y2,x] = True
    """
############## Source detector　か Peak Detector #######

    mean, median, std = sigma_clipped_stats(data, sigma=3.0)
    #tbl = find_peaks(data, threshold=50.*std, box_size=11)
    #tbl['peak_value'].info.format = '%.8g'
    #positions_peak = (tbl['x_peak'], tbl['y_peak'])
    
    
    daofind = DAOStarFinder(fwhm=3.0, threshold=50.*std)    ##############ここの設定をみる必要あり
    sources = daofind(data - median)                        ### souurce detection じゃなくて peak detection
    for col in sources.colnames:                            ###のほうがいいかもしれない
        sources[col].info.format = '%.8g'  # for consistent table output
    positions = (sources['xcentroid'], sources['ycentroid'])
    
###################################### xml ファイルの読み込み #######
    with open(filename_xml[i]) as f:
        l = f.readlines()
        for j in range(len(l)):
            m = l[j]
            if m == b:
                del_start = j + 1
            elif m == c:
                del_end = j
        l = l[del_start:del_end]
        xml = l[0]
        xml = re.split('<TR><TD>|</TD><TD>|</TD></TR>', xml)
        del xml[0],xml[2:6],xml[3:12],xml[5:]
        for k in l:
            s = re.split('<TR><TD>|</TD><TD>|</TD></TR>', k)
            del s[0],s[2:6],s[3:12],s[5:]
            if s != ['', '', '', '','']:
                xml_list = np.vstack((xml_list, s))
    
    xml_list = np.delete(xml_list,0, 0)
    #print xml_list
    source_ID = xml_list[:,2]
    source_ID = source_ID.reshape(len(source_ID),1)
    xml_list = xml_list.astype(np.float64)
    pixel_coordinates = np.arange(2).reshape(1,2)
    for k in range(len(xml_list)):
                pixel_coord = wcs.all_world2pix(xml_list[k,0], xml_list[k,1], 1)
                pixel_coordinates = np.vstack((pixel_coordinates, pixel_coord))
    pixel_coordinates = np.delete(pixel_coordinates,0, 0)
    #xml_list = np.hstack((xml_list,pixel_coordinates,source_ID))
    xml_list = np.hstack((xml_list,pixel_coordinates))
    xml_list = np.delete(xml_list,2,axis=1)
    
    #x座標がフレーム外のindex行をxmkから削除
    #apertureが取れるように10pix余裕とる
    index = np.where((xml_list[:,4]>490.)|(xml_list[:,4]<10.)|\
                    (xml_list[:,5]>272.) | (xml_list[:,5]<10.))
    xml_list = np.delete(xml_list,index, 0)
    source_ID = np.delete(source_ID,index,0)
    #xml_list = np.hstack((xml_list,source_ID))
    #xml_all = np.vstack((xml_all,xml_list))
#####################################
# Source Extractor と比較
    pix_sextractor = np.array(positions)
    pix_sextractor = pix_sextractor.transpose()
    use_index = []
    for m in range(len(pix_sextractor)):
    #for m in range(1):
        index = abs(xml_list[:,4] - pix_sextractor[m,0])+\
                abs(xml_list[:,5] - pix_sextractor[m,1])
        if index.min() < 4.0:#候補がないやつを除く
            a = index.argmin()
            use_index = np.hstack((use_index,a))
            use_index = use_index.astype(np.int64)
            pixel_sextractor = np.vstack((pixel_sextractor,pix_sextractor[m,:]))
    
    pixel_sextractor = np.delete(pixel_sextractor,0,0)
    xml_list = xml_list[use_index,:]
    source_ID = source_ID[use_index,:]
    xml_list = np.hstack((xml_list,source_ID,pixel_sextractor))
    xml_all = np.vstack((xml_all,xml_list))
    xml_all = np.delete(xml_all,0,0)
    
    plt.imshow(data, cmap = 'gray', vmin = 0, vmax = 400, interpolation = 'none')
    plt.ylim(0, 282)
    #plt.scatter(xml_all[:,3],xml_all[:,4],alpha = 0.5)
    #rectangular.plot(color='yellow', lw=0.5, alpha=5.)
    apertures = CircularAperture(positions_peak, r=5.)
    #apertures = CircularAperture(positions, r=5.)
    apertures.plot(color='green', lw=1.5, alpha=0.5)
    #pdf = PdfPages("filename.pdf") #図をpdfに出力
    #pdf.savefig()
    #pdf.close()
    plt.show()
    

### 今はsource_IDとxml_listをhstackさせちゃうけど　sextractor でどうせ
##まだできなくなるから,source_IDを別にしておく
    columns = ["Ra_catalog", "Dec_catalog", "GMag", "Gmag_err", "x_astropy",\
               "y_astropy", "source_ID", "x_sextract", "y_sextract"]
    df = pd.DataFrame(data = xml_all, columns = columns)
    df.to_csv(output_filename[i])
clear_output()
print "Complete!"

Complete!
